# Feature Engineering

In [1]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random as rd
%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [2]:
from scipy.stats.mstats import winsorize

## Carga de datos

Se cargan las columnas con formatos adecuados según el tipo de datos.  
Se usa como índice building_id para train_values y train_labels  

In [3]:
# Train values
# Levantar los datos de cada edificio/casa
df = pd.read_csv('datos/train_values.csv',nrows=0)
columnas = df.columns.to_list()
datatypes = {
    **{columnas[0]:np.int32}\
    , **{column:np.int16 for column in columnas[1:8] + columnas[27:28]}\
    , **{column:'category' for column in columnas[8:15] + columnas[26:27]}\
    , **{column:np.bool for column in columnas[15:26] + columnas[28:39]}
}
train_values = pd.read_csv('datos/train_values.csv',dtype=datatypes)
train_values = train_values.set_index("building_id")
columnas = train_values.columns.to_list()

In [4]:
#Train labels
train_labels = pd.read_csv('datos/train_labels.csv',index_col='building_id')

## Información de los datasets

In [5]:
train_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 38 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   geo_level_1_id                          260601 non-null  int16   
 1   geo_level_2_id                          260601 non-null  int16   
 2   geo_level_3_id                          260601 non-null  int16   
 3   count_floors_pre_eq                     260601 non-null  int16   
 4   age                                     260601 non-null  int16   
 5   area_percentage                         260601 non-null  int16   
 6   height_percentage                       260601 non-null  int16   
 7   land_surface_condition                  260601 non-null  category
 8   foundation_type                         260601 non-null  category
 9   roof_type                               260601 non-null  category
 10  ground_floor_type          

In [6]:
train_values.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,False,False,False,False,False,False,False,False,False,False
28830,8,900,2812,2,10,8,7,o,r,n,...,False,False,False,False,False,False,False,False,False,False
94947,21,363,8973,2,10,5,5,t,r,n,...,False,False,False,False,False,False,False,False,False,False
590882,22,418,10694,2,10,6,5,t,r,n,...,False,False,False,False,False,False,False,False,False,False
201944,11,131,1488,3,30,8,9,t,r,n,...,False,False,False,False,False,False,False,False,False,False


In [7]:
train_labels.head()

,damage_grade
building_id,
802906,3
28830,2
94947,3
590882,2
201944,3


# Aplicamos transformaciones

In [8]:
# for combining the preprocess with model training
from sklearn.pipeline import Pipeline

# for optimizing the hyperparameters of the pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import FeatureUnion

In [9]:
class DataframeFunctionTransformer():
    def __init__(self, func):
        self.func = func

    def transform(self, input_df, **transform_params):
        return self.func(input_df)

    def fit(self, X, y=None, **fit_params):
        return self

# this function takes a dataframe as input and
# returns a modified version thereof
def select_columns_iloc(input_df,range_columns):
    output_df = input_df.iloc[:,range_columns[0]:range_columns[1]]
    return output_df

def drop_columns(input_df,columns):
    output_df = input_df.copy()
    for column_name in columns:
        output_df = output_df.drop(labels=column_name,axis=1)
    return output_df

def winsorize_columns(input_df,columns_to_winsorize,ranges_winsorize):
    for i in range(len(columns_to_winsorize)):
        columna_winsorize = winsorize(input_df[columns_to_winsorize[i]],ranges_winsorize[i])
        input_df[columns_to_winsorize[i]] = columna_winsorize
    return input_df

def get_dummies_transform(input_df):
    return pd.get_dummies(input_df)

#auxiliar
def nombre_nueva_mean_col(col_name,damage_grade):
    damage_names = {1:'low',2:'medium',3:'high'}
    return 'mean_'+damage_names[damage_grade]+'_damage_'+col_name

#generalizar para una columna
def agregar_col_mean_daño_por_columna(input_df,labels_df,col_name,damage_grade):
    """Agrega una columna llamada mean_damage_'col_name' con el mean damage para el daño 'damage_grade'"""
    work_df = input_df[[col_name]].copy()
    work_df = work_df.join(labels_df,on='building_id',how='inner')
    work_df['count_damage_i'] = work_df['damage_grade'] == damage_grade
    new_col_name = nombre_nueva_mean_col(col_name,damage_grade)
    input_df[new_col_name] = work_df.groupby(col_name).transform('mean')['count_damage_i']
    return input_df


In [28]:
#Ajustes iniciales
pipeline_inicial = Pipeline([
    ("drop_has_sec_variants", FunctionTransformer(select_columns_iloc,kw_args={'range_columns':(0,28)})),
    ("drop_plan_config", FunctionTransformer(drop_columns,kw_args={'columns': ['plan_configuration']})),
    ("winsorize", FunctionTransformer(winsorize_columns,kw_args={'columns_to_winsorize': ['age','area_percentage','height_percentage'],
                                                                'ranges_winsorize': [(0,0.03),(0,0.03),(0,0.06)]})),
])
# Probar al final si sacarlos obtengo mejor score.

In [29]:
#Mean col geo
steps_daños = []
for geo_level_id in range(1,4):
    for damage_grade in range(1,4):
        nombre_step = 'add_damage_'+str(damage_grade)+'_geo_'+str(geo_level_id)
        args = {'labels_df': train_labels,'col_name': 'geo_level_'+str(geo_level_id)+'_id','damage_grade':damage_grade}
        steps_daños.append((nombre_step,FunctionTransformer(agregar_col_mean_daño_por_columna,kw_args=args)))

In [30]:
pipeline_daños = Pipeline(steps=steps_daños)
#pipeline_daños

In [31]:
#Mean col area
steps_area = []
for damage_grade in range(1,4):
    nombre_step = 'add_damage_'+str(damage_grade)+'_height'
    args = {'labels_df': train_labels,'col_name': 'area_percentage','damage_grade':damage_grade}
    steps_area.append((nombre_step,FunctionTransformer(agregar_col_mean_daño_por_columna,kw_args=args)))

In [32]:
pipeline_area= Pipeline(steps=steps_area)

### Aplico transformaciones

In [33]:
train_values_transformado = pipeline_inicial.fit_transform(train_values)

In [34]:
train_values_transformado = pipeline_daños.fit_transform(train_values_transformado)

In [35]:
train_values_transformado = pipeline_area.fit_transform(train_values_transformado)

In [36]:
train_values_transformado = pd.get_dummies(train_values_transformado)

In [37]:
train_values_transformado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 60 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   geo_level_1_id                          260601 non-null  int16  
 1   geo_level_2_id                          260601 non-null  int16  
 2   geo_level_3_id                          260601 non-null  int16  
 3   count_floors_pre_eq                     260601 non-null  int16  
 4   age                                     260601 non-null  int16  
 5   area_percentage                         260601 non-null  int16  
 6   height_percentage                       260601 non-null  int16  
 7   has_superstructure_adobe_mud            260601 non-null  bool   
 8   has_superstructure_mud_mortar_stone     260601 non-null  bool   
 9   has_superstructure_stone_flag           260601 non-null  bool   
 10  has_superstructure_cement_mortar_stone 

XGBoost

In [38]:
import xgboost as xgb
import time
from datetime import datetime
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import sys

## Utilizo los mejores parámetros

In [89]:
xgb_model = xgb.XGBClassifier(objective="multi:softprob",
                              eval_metric='mlogloss',
                              max_depth=9,
                              n_estimators= 340,
                              eta=0.1,
                              gamma=3,
                              subsample= 0.9,
                              max_delta_step= 0,
                              colsample_bytree= 0.9,
                              min_child_weight=2,  
                              )

In [90]:
start_time = time.time()
train_values_subset = train_values_transformado
#scores_best_norm = cross_val_score(xgb_model, train_values_transformado, train_labels.values.ravel())
xgb_model.fit(train_values_subset,train_labels.values.ravel() )
print("--- Duración: %s segundos ---" % (time.time() - start_time))
#print(scores_best_norm)
#print(scores_best_norm.mean())

--- Duración: 700.8613498210907 segundos ---


In [91]:
pprint(xgb_model.get_params())

Pretty printing has been turned ON


In [92]:
#Score a mejorar 0.7931..
y_pred = xgb_model.predict(train_values_subset)
f1_score(train_labels, y_pred, average='micro')

0.7815894797026872

In [93]:
# import pickle
# filename = 'modelos/xgb_modelo_7444.sav'
# pickle.dump(xgb_model, open(filename, 'wb'))

**Pruebas raras**:
* Si tiene la misma cantidad de columnas con el cual se entrenó el algoritmo -> lo va a predecir (Ojo)  

Registro de scores (random_state = 99, con objective=multi:softmax y eval_metric= mlogloss sin nada más):  
**GENERALES**
* Con todos los features y one-hot-encoding -> 0.7442
* Con todos los features (OHE) y normalización (StandardScaler) -> 0.7452
* Con todos los features (OHE) y normalización (MinMaxScaler) -> 0.7452  



**Eliminación de columnas**  
* Sin plan_configuration : 0.7447
* Sin has_secondary específicos y sin plan_configuration : Score: 0.7467 Score Test: 0.7304
* Sin has_secondary (ninguno) -> 0.7440
* Sin plan configuration -> 0.7442
* Sin has_secondary (ninguno) y sin legal_ownership_status-> 0.7437
* Sin has_secondary específicos -> 0.743922
* Sin has_secondary específicos y sin plan configuration ->0.7456
* Lo mismo que el anterior pero ahora sin plan_configuration -> 0.7412  

**Sin has_secondary específicos ni plan_configuration- Objetivo>0.7467** (Mejorar no hacer nada):  
* Con winsorizing para age:0.05 , area_percentage:0.06 , height_percentage:0.02. Score entr:0.7457
* Con winsorizing para age:0.01 , area_percentage:0.06 , height_percentage:0.02. Score entr:0.7454 -> legal_ownership_status
* Con winsorizing para age:0.03 , area_percentage:0.03 , height_percentage:0.06. Score entr:0.7473 Score Test:0.7307

**Pruebo sin sacar columnas**
* Con winsorizing para age:0.03 , area_percentage:0.03 , height_percentage:0.06. Score entr:0.7448

**Target encoding damage**
* geo_level_1_id -> score:0.7463
* geo_level_2_id -> score:0.7529
* geo_level_3_id -> score:0.7689
* geo_level_1_id + geo_level_2_id: 0.7531
* geo_level_1_id + geo_level_2_id + geo_level_3_id -> Train score:  0.7690. Test Score: 0.7409
* geo_level_id todos los daños y todos los id: 0.7766, Test Score: 0.7444
* mean damages superstructure->: 0.7460
* geo_level_id (todos) + superstructure (todos) -> 0.7768, Test Score: 0.7428

**Randomized Search**
ranking 1-> Score test: 0.7468
ranking 2-> Score test: 0.74

## Predecir el set de test de DrivenData

Tenemos que predecir nuevos valores de los que desconocemos sus labels o daños.  
Solo tenemos sus columnas o features. Vamos a utilizar nuestro modelo para predecir.

In [94]:
test_values = pd.read_csv('datos/test_values.csv', index_col='building_id')

In [95]:
test_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86868 entries, 300051 to 501372
Data columns (total 38 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   geo_level_1_id                          86868 non-null  int64 
 1   geo_level_2_id                          86868 non-null  int64 
 2   geo_level_3_id                          86868 non-null  int64 
 3   count_floors_pre_eq                     86868 non-null  int64 
 4   age                                     86868 non-null  int64 
 5   area_percentage                         86868 non-null  int64 
 6   height_percentage                       86868 non-null  int64 
 7   land_surface_condition                  86868 non-null  object
 8   foundation_type                         86868 non-null  object
 9   roof_type                               86868 non-null  object
 10  ground_floor_type                       86868 non-null  object
 

El set de test tiene que tener el MISMO formato que usamos para entrenar.  
Es decir, si le hicimos algunas modificaciones en las columnas al set de entrenamiento, también tenemos que hacerlas al set de test.  
En este caso, solo seleccionamos algunas columnas y utilizamos One-Hot-Encoding para las variables categóricas (sget_dummies)

Trasformar el set de test

In [96]:
def agregar_columna_mean_damage(input_df,column_name,train_values,train_labels,damage_grade):
    """Recibe un df y calcula la media de daño 'damage_grade' para cada valor distinto de 'column_name' 
    """
    # Copio el df inicial para calcular mean sin afectar al input
    df = train_values.copy()
    df = df[[column_name]]
    df = df.join(train_labels,on='building_id',how='inner')
    
    # Se le agrega una columna a df con 1 si el edificio recibio daño del tipo 'damage_grade',0 si no
    df['count_damage_i'] = df['damage_grade'] == damage_grade
    
    # mean damage es un df con indice 'column_name' y como unica columna 'count_high_damage'
    mean_damage = df.groupby(column_name).agg('mean')[['count_damage_i']]
    
    # output df es el df inicial pero ahora con una nueva columna con los daños medios para cada elemento
    # de la columna 'column_name'
    output_df = input_df.join(mean_damage,on=column_name,how='left',)
    nombre_nueva_col = nombre_nueva_mean_col(column_name,damage_grade)
   
    # ahora esa columna tiene el nombre 'nombre_nueva_col'
    output_df = output_df.rename(columns={'count_damage_i': nombre_nueva_col})
    
    # manejo de nulls -> agrego la media global de ese nombre_nueva_col si es null 
    mean_global = output_df[nombre_nueva_col].mean()
    output_df[nombre_nueva_col] = output_df[nombre_nueva_col].fillna(value=mean_global)
    return output_df


In [97]:
### creo los pasos para que el pipeline cree las columnas de mean daño para cada daño y para cada geo_level_id
steps_daños_test = []
for geo_level_id in range(1,4):
    for damage_grade in range(1,4):
        nombre_step = 'add_damage_'+str(damage_grade)+'_geo_'+str(geo_level_id)
        args = {'train_values':train_values,'train_labels': train_labels,'column_name': 'geo_level_'+str(geo_level_id)+'_id','damage_grade':damage_grade}
        steps_daños_test.append((nombre_step,FunctionTransformer(agregar_columna_mean_damage,kw_args=args)))

In [98]:
pipeline_daños_test = Pipeline(steps=steps_daños_test)
#pipeline_daños_test

In [99]:
steps_area_test = []
for damage_grade in range(1,4):
    nombre_step = 'add_damage_'+str(damage_grade)+'_height'
    args = {'train_values':train_values,'train_labels': train_labels,'column_name': 'area_percentage','damage_grade':damage_grade}
    steps_area_test.append((nombre_step,FunctionTransformer(agregar_columna_mean_damage,kw_args=args)))

In [100]:
pipeline_area_test = Pipeline(steps=steps_area_test)


In [101]:
## Solo para un caso de los que hice.. según la modificación se tiene que transformar..
test_values_subset = pipeline_inicial.fit_transform(test_values)

In [102]:
test_values_subset = pipeline_daños_test.fit_transform(test_values_subset)

In [103]:
test_values_subset = pipeline_area_test.fit_transform(test_values_subset)

In [104]:
test_values_subset = pd.get_dummies(test_values_subset)

In [105]:
test_values_subset.columns.to_list() == train_values_transformado.columns.to_list()

True

In [106]:
test_values_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86868 entries, 300051 to 501372
Data columns (total 60 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   geo_level_1_id                          86868 non-null  int64  
 1   geo_level_2_id                          86868 non-null  int64  
 2   geo_level_3_id                          86868 non-null  int64  
 3   count_floors_pre_eq                     86868 non-null  int64  
 4   age                                     86868 non-null  int64  
 5   area_percentage                         86868 non-null  int64  
 6   height_percentage                       86868 non-null  int64  
 7   has_superstructure_adobe_mud            86868 non-null  int64  
 8   has_superstructure_mud_mortar_stone     86868 non-null  int64  
 9   has_superstructure_stone_flag           86868 non-null  int64  
 10  has_superstructure_cement_mortar_stone  86868 non-nu

Predicimos con modelo.predict(set_test_transformado) Le pasamos los valores del set de test.

In [113]:
predictions = xgb_model.predict(test_values_subset)

Guardamos el submit. Lo pasamos a Pandas y de ahí a CSV (por tema de formato).  
Esto es lo que se sube a la página de DrivenData

In [114]:
submission_format = pd.read_csv('datos/submission_format.csv', index_col='building_id')

In [115]:
my_submission = pd.DataFrame(data=predictions,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [116]:
my_submission.head()

,damage_grade
building_id,
300051,3
99355,2
890251,3
745817,1
421793,3


In [117]:
# Cambiar el nombre al submission para comparar resultados es buena idea..
my_submission.to_csv('submissions/last_submission.csv')

Una mirada adentro del csv

In [118]:
!head 'submissions/last_submission.csv'

building_id,damage_grade
300051,3
99355,2
890251,3
745817,1
421793,3
871976,2
691228,2
896100,3
343471,2
